# Sprint 機械学習フロー

## 【問題1】クロスバリデーション

事前学習期間では検証データをはじめに分割しておき、それに対して指標値を計算することで検証を行っていました。（ホールドアウト法）しかし、分割の仕方により精度は変化します。実践的には クロスバリデーション（交差検証） を行います。分割を複数回行い、それぞれに対して学習と検証を行う方法です。複数回の分割のためにscikit-learnにはKFoldクラスが用意されています。


事前学習期間の課題で作成したベースラインモデルに対してKFoldクラスによるクロスバリデーションを行うコードを作成し実行してください。


In [15]:
import pandas as pd 
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
from scipy.stats import pearsonr
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
from sklearn.metrics import mean_squared_error
%matplotlib inline 

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=FutureWarning) 
warnings.filterwarnings("ignore", category=UserWarning) 

In [16]:
df = pd.read_csv("dataset/application_train.csv")

In [17]:
# int/floatとobjectを分離
df_tmp = df
list_str=[] 

for i in df_tmp.columns:
    if df_tmp[i].dtypes == 'O':
        df_tmp = df_tmp.drop([i], axis = 1)
        list_str.append(i)

# 欠損値を平均値で埋める
df_tmp = df_tmp.fillna(df_tmp.mean())

# objectをダミー変数に変換
df_str = df[list_str]
df_str_dummy = pd.get_dummies(df_str)

#object(ダミー変数に変換)と、int/float(欠損値を０で置換)を結合
df_mix = pd.concat([df_tmp, df_str_dummy], axis=1)
# display(df_mix.describe())
# display(df_mix.info())

# TARGETとの相関係数の絶対値が0.06以上のデータを抽出
list_i = []
for i in df_mix.columns:
    a, b = pearsonr(df_mix[i], df_mix["TARGET"])
    if 0.04 < abs(a) :
        list_i.append(i)
        # print("{}, {:.2f}".format(i, a))

# 対象とする特徴量のリスト
features_list = list_i

# 特徴量と目的変数を格納
X = df_mix[features_list].drop(['TARGET'], axis=1)
y = df_mix[['TARGET']]

# ndarrayに変換
X = X.values
y = y.values

In [18]:
#データの分割
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, random_state=0)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((230633, 24), (76878, 24), (230633, 1), (76878, 1))

In [19]:
# クロスバリデーション
from sklearn.model_selection import KFold
kf = KFold(n_splits=10, shuffle=True, random_state=0)
kf.get_n_splits(X)
print(kf)
score_list = []

for train_index, test_index in kf.split(X):
    # print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # 標準化
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train_std = scaler.transform(X_train)
    X_test_std = scaler.transform(X_test)

    #ロジスティック回帰
    from sklearn.linear_model import LogisticRegression
    logistic = LogisticRegression(random_state=0, solver='lbfgs')
    logistic.fit(X_train_std, y_train.ravel())
    y_predict_logistic = logistic.predict_proba(X_test_std)[:,1]

    # 評価
    score = roc_auc_score(y_test, y_predict_logistic)
    score_list.append(score)
    print("スコア：{}".format(score))

print("平均：{}".format(sum(score_list) / len(score_list)))

KFold(n_splits=10, random_state=0, shuffle=True)
スコア：0.735270160509338
スコア：0.7372116053732689
スコア：0.7391705073716905
スコア：0.7378840034474046
スコア：0.7364471171088518
スコア：0.7315383088037155
スコア：0.7264793591004837
スコア：0.7402081540113885
スコア：0.7381263444443619
スコア：0.7357412882375106
平均：0.7358076848408014


## 【問題2】グリッドサーチ

これまで分類器のパラメータには触れず、デフォルトの設定を使用していました。パラメータの詳細は今後のSprintで学んでいくことになります。機械学習の前提として、パラメータは状況に応じて最適なものを選ぶ必要があります。最適なパラメータを探していくことを パラメータチューニング と呼びます。パラメータチューニングをある程度自動化する単純な方法としては グリッドサーチ があります。


scikit-learnのGridSearchCVを使い、グリッドサーチを行うコードを作成してください。そして、ベースラインモデルに対して何らかしらのパラメータチューニングを行なってください。どのパラメータをチューニングするかは、使用した手法の公式ドキュメントを参考にしてください。

In [20]:
#グリットサーチを行う
from sklearn.model_selection import GridSearchCV
 
param_grid = {'C' : [0.001, 0.01, 0.1, 1, 10, 100]}
 
GridSearch = GridSearchCV(logistic,param_grid,cv=6, scoring = "roc_auc")
GridSearch.fit(X_train_std,y_train.ravel())

GridSearchCV(cv=6, error_score='raise-deprecating',
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='warn',
                                          n_jobs=None, penalty='l2',
                                          random_state=0, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='roc_auc', verbose=0)

In [21]:
#グリットサーチの結果において、最適なパラメータを表示する
GridSearch.best_estimator_

LogisticRegression(C=100, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [22]:
#評価の実行(グリットサーチを使用)
GridSearch.score(X_test_std, y_test.ravel())

0.735812643339746

## 【問題3】Kaggle Notebooksからの調査

KaggleのNotebooksから様々なアイデアを見つけ出して、列挙してください。

- 手法の追加（LightGBM、XGBoost、CatBoost、SHAP）
- 特徴量の作成
- 外れ値の除外

## 【問題4】高い汎化性能のモデル作成

問題3で見つけたアイデアと、独自のアイデアを組み合わせ高い汎化性能のモデル作りを進めてください。


その過程として、何を行うことで、クロスバリデーションの結果がどの程度変化したかを表にまとめてください。

### KaggleのNotebooksで採用されていることが多かったLightGBMを採用

In [23]:
# LightGBM
import lightgbm as lgbm
 
lgbm = lgbm.LGBMClassifier()
lgbm.fit(X_train_std, y_train.ravel())
y_predict_lgbm = lgbm.predict_proba(X_test_std)[:,1]

# 評価
score = roc_auc_score(y_test, y_predict_lgbm)
score_list.append(score)
print("スコア：{}".format(score))

# grid_param ={'n_estimators':[1000,2000],'max_depth':[4,8,16]}
# bst_gs_cv = GridSearchCV(
#             lgb.LGBMRegressor(),
#             grid_param,
#             cv = KFold(n_splits=3, shuffle=True),
#             scoring = 'roc_auc',
#             verbose = 0
#             )
# bst_gs_cv.fit(
#             train2, 
#             target,
#             verbose = 0
#             )

スコア：0.7468125867691727


ロジスティック回帰よりも良いスコアを算出。  
さらにグリッドサーチを行い、パラメータを調整する。

In [34]:
#グリットサーチを行う
from sklearn.model_selection import GridSearchCV

# lgbm_params = {"learning_rate": [0.001, 0.01, 0.1],
#              "n_estimators": [200, 500, 100],
#              "max_depth":[1,2,35,8]}

lgbm_params = {"n_estimators": [10, 100],
             "max_depth":[3,5,7,10]}

# lgbm_cv_model = GridSearchCV(lgbm,lgbm_params, cv = 10, n_jobs = -1, verbose = 4).fit(X_train_std, y_train)
# lgbm_cv_model.best_params_

GridSearch = GridSearchCV(lgbm,lgbm_params,cv=6, scoring = "roc_auc")
GridSearch.fit(X_train_std,y_train.ravel())

KeyboardInterrupt: 

In [25]:
#グリットサーチの結果において、最適なパラメータを表示する
GridSearch.best_estimator_

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=5,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [28]:
#評価の実行(test)
GridSearch.score(X_test_std, y_test.ravel())

0.7465795188284784

スコアの変動はほぼ無し。
検証用データをkaggleに提出する。

In [91]:
# テストデータの読み込み
df_test = pd.read_csv("dataset/application_test.csv")

# int/floatとobjectを分離
df_tmp = df_test
list_str=[] 

for i in df_tmp.columns:
    if df_tmp[i].dtypes == 'O':
        df_tmp = df_tmp.drop([i], axis = 1)
        list_str.append(i)

# 欠損値を平均値で埋める
df_tmp = df_tmp.fillna(df_tmp.mean())

# int/floatの欠損値を0で置換して確認
# df_tmp = df_tmp.replace(np.nan,0)

# objectをダミー変数に変換
df_str = df_test[list_str]
df_str_dummy = pd.get_dummies(df_str)
# df_str_dummy.describe()

In [102]:
#object(ダミー変数に変換)と、int/float(欠損値を０で置換)を結合
df_mix = pd.concat([df_tmp, df_str_dummy], axis=1)

# ndarrayに変換
X_test = df_mix[features_list[1:]].values  #トレーニングで作成したリストからTargetを除外
# y = y.values

In [103]:
# 標準化
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_test)
X_test_std = scaler.transform(X_test)

In [104]:
# LightGBM
import lightgbm as lgbm
lgbm = GridSearch.best_estimator_
lgbm.fit(X_train_std, y_train.ravel())
y_predict_lgbm = lgbm.predict_proba(X_test_std)[:,1]
y_predict_lgbm

array([0.02672011, 0.07910463, 0.01514794, ..., 0.04834459, 0.03233199,
       0.11776367])

In [105]:
sample_submission = pd.read_csv("dataset/sample_submission.csv")
display(sample_submission.shape)
display(sample_submission)

(48744, 2)

,SK_ID_CURR,TARGET
0,100001,0.5
1,100005,0.5
2,100013,0.5
3,100028,0.5
4,100038,0.5
...,...,...
48739,456221,0.5
48740,456222,0.5
48741,456223,0.5
48742,456224,0.5


In [ ]:
sample_submission["TARGET"] = y_predict_lgbm
display(sample_submission)

In [97]:
sample_submission.to_csv('dataset/submission5.csv', index=False)

kaggleのスコアは**0.72233**という結果で、ロジスティック回帰のデフォルトとパフォーマンスは変わらなかった。

### 特徴量の修正を行う

特徴量の外れ値削除と新規の特徴量を作成し、検証を行う。

In [109]:
df = pd.read_csv("dataset/application_train.csv")

# 特徴量の修正
df = df[df['CODE_GENDER'] != 'XNA']
df['DAYS_EMPLOYED'].replace(365243, np.nan, inplace = True)
df['NEW_DAYS_EMPLOYED_PERC'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']
df['NEW_INCOME_CREDIT_PERC'] = df['AMT_INCOME_TOTAL'] / df['AMT_CREDIT']
df['NEW_INCOME_PER_PERSON'] = df['AMT_INCOME_TOTAL'] / df['CNT_FAM_MEMBERS']
df['NEW_ANNUITY_INCOME_PERC'] = df['AMT_ANNUITY'] / df['AMT_INCOME_TOTAL']
df['NEW_PAYMENT_RATE'] = df['AMT_ANNUITY'] / df['AMT_CREDIT']

# int/floatとobjectを分離
df_tmp = df
list_str=[] 

for i in df_tmp.columns:
    if df_tmp[i].dtypes == 'O':
        df_tmp = df_tmp.drop([i], axis = 1)
        list_str.append(i)

# 欠損値を平均値で埋める
df_tmp = df_tmp.fillna(df_tmp.mean())

# objectをダミー変数に変換
df_str = df[list_str]
df_str_dummy = pd.get_dummies(df_str)

#object(ダミー変数に変換)と、int/float(欠損値を０で置換)を結合
df_mix = pd.concat([df_tmp, df_str_dummy], axis=1)
# display(df_mix.describe())
# display(df_mix.info())

# TARGETとの相関係数の絶対値が0.04以上のデータを抽出
list_i = []
for i in df_mix.columns:
    a, b = pearsonr(df_mix[i], df_mix["TARGET"])
    if 0.04 < abs(a) :
        list_i.append(i)
        # print("{}, {:.2f}".format(i, a))

# 対象とする特徴量のリスト
features_list = list_i

In [110]:
features_list

['TARGET',
 'DAYS_BIRTH',
 'DAYS_EMPLOYED',
 'DAYS_REGISTRATION',
 'DAYS_ID_PUBLISH',
 'FLAG_EMP_PHONE',
 'REGION_RATING_CLIENT',
 'REGION_RATING_CLIENT_W_CITY',
 'REG_CITY_NOT_LIVE_CITY',
 'REG_CITY_NOT_WORK_CITY',
 'EXT_SOURCE_1',
 'EXT_SOURCE_2',
 'EXT_SOURCE_3',
 'DAYS_LAST_PHONE_CHANGE',
 'FLAG_DOCUMENT_3',
 'NEW_DAYS_EMPLOYED_PERC',
 'CODE_GENDER_F',
 'CODE_GENDER_M',
 'NAME_INCOME_TYPE_Pensioner',
 'NAME_INCOME_TYPE_Working',
 'NAME_EDUCATION_TYPE_Higher education',
 'NAME_EDUCATION_TYPE_Secondary / secondary special',
 'OCCUPATION_TYPE_Laborers',
 'ORGANIZATION_TYPE_XNA',
 'HOUSETYPE_MODE_block of flats',
 'EMERGENCYSTATE_MODE_No']

In [118]:

# 特徴量と目的変数を格納
X = df_mix[features_list].drop(['TARGET'], axis=1)
y = df_mix[['TARGET']]

# ndarrayに変換
X = X.values
y = y.values

#データの分割
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, random_state=0)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

# 標準化
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train_std = scaler.transform(X_train)
X_test_std = scaler.transform(X_test)

# LightGBM
import lightgbm as lgbm
 
lgbm = lgbm.LGBMClassifier()
lgbm.fit(X_train_std, y_train.ravel())
y_predict_lgbm = lgbm.predict_proba(X_test_std)[:,1]

# 評価
score = roc_auc_score(y_test, y_predict_lgbm)
score_list.append(score)
print("スコア：{}".format(score))

スコア：0.7477431453788022


In [119]:
#グリットサーチを行う
from sklearn.model_selection import GridSearchCV

# lgbm_params = {"learning_rate": [0.001, 0.01, 0.1],
#              "n_estimators": [200, 500, 100],
#              "max_depth":[1,2,35,8]}

lgbm_params = {"n_estimators": [10, 100],
             "max_depth":[3,5,7,10]}

# lgbm_cv_model = GridSearchCV(lgbm,lgbm_params, cv = 10, n_jobs = -1, verbose = 4).fit(X_train_std, y_train)
# lgbm_cv_model.best_params_

GridSearch = GridSearchCV(lgbm,lgbm_params,cv=6, scoring = "roc_auc")
GridSearch.fit(X_train_std,y_train.ravel())

GridSearchCV(cv=6, error_score='raise-deprecating',
             estimator=LGBMClassifier(boosting_type='gbdt', class_weight=None,
                                      colsample_bytree=1.0,
                                      importance_type='split',
                                      learning_rate=0.1, max_depth=-1,
                                      min_child_samples=20,
                                      min_child_weight=0.001,
                                      min_split_gain=0.0, n_estimators=100,
                                      n_jobs=-1, num_leaves=31, objective=None,
                                      random_state=None, reg_alpha=0.0,
                                      reg_lambda=0.0, silent=True,
                                      subsample=1.0, subsample_for_bin=200000,
                                      subsample_freq=0),
             iid='warn', n_jobs=None,
             param_grid={'max_depth': [3, 5, 7, 10], 'n_estimators': [10, 100]},
   

In [121]:
#評価の実行(test)
GridSearch.score(X_test_std, y_test.ravel())

0.7482411932083337

 LightGBMによるスコアアップは間違いなさそうだが、クロスバリデーションによる汎化性能の向上という面に関しては明確な差はみられなかった。

|検証内容|訓練データ|kaggle Private Score|kaggle Public Score|
|:-|:-|:-|:-|
|ロジスティック回帰|0.736844772|0.72034|0.7245|
|ロジスティック回帰/クロスバリデーション|0.735807685|
|ロジスティック回帰/グリッドサーチ|0.735812643|
|LightGBM|0.746812587|
|LightGBM/グリッドサーチ|0.746579519|0.72233|0.72431|
|LightGBM/特徴量の修正|0.748241193|0.72515|0.73109|


## 【問題5】最終的なモデルの選定

### 特徴量の作成、削除を行った上でkaggleに登録する

In [123]:
# テストデータの読み込み
df_test = pd.read_csv("dataset/application_test.csv")

# 特徴量の修正
df = df_test
df = df[df['CODE_GENDER'] != 'XNA']
df['DAYS_EMPLOYED'].replace(365243, np.nan, inplace = True)
df['NEW_DAYS_EMPLOYED_PERC'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']
df['NEW_INCOME_CREDIT_PERC'] = df['AMT_INCOME_TOTAL'] / df['AMT_CREDIT']
df['NEW_INCOME_PER_PERSON'] = df['AMT_INCOME_TOTAL'] / df['CNT_FAM_MEMBERS']
df['NEW_ANNUITY_INCOME_PERC'] = df['AMT_ANNUITY'] / df['AMT_INCOME_TOTAL']
df['NEW_PAYMENT_RATE'] = df['AMT_ANNUITY'] / df['AMT_CREDIT']
df_test = df

# int/floatとobjectを分離
df_tmp = df_test
list_str=[] 

for i in df_tmp.columns:
    if df_tmp[i].dtypes == 'O':
        df_tmp = df_tmp.drop([i], axis = 1)
        list_str.append(i)

# 欠損値を平均値で埋める
df_tmp = df_tmp.fillna(df_tmp.mean())

# int/floatの欠損値を0で置換して確認
# df_tmp = df_tmp.replace(np.nan,0)

# objectをダミー変数に変換
df_str = df_test[list_str]
df_str_dummy = pd.get_dummies(df_str)
# df_str_dummy.describe()

In [124]:
#object(ダミー変数に変換)と、int/float(欠損値を０で置換)を結合
df_mix = pd.concat([df_tmp, df_str_dummy], axis=1)

# ndarrayに変換
X_test = df_mix[features_list[1:]].values  #トレーニングで作成したリストからTargetを除外
# y = y.values

In [125]:
# 標準化
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_test)
X_test_std = scaler.transform(X_test)

In [126]:
# LightGBM
import lightgbm as lgbm
lgbm = GridSearch.best_estimator_
lgbm.fit(X_train_std, y_train.ravel())
y_predict_lgbm = lgbm.predict_proba(X_test_std)[:,1]
y_predict_lgbm

array([0.04416534, 0.08876724, 0.0176113 , ..., 0.05274743, 0.03871433,
       0.14277076])

In [127]:
sample_submission = pd.read_csv("dataset/sample_submission.csv")
display(sample_submission.shape)
display(sample_submission)

(48744, 2)

,SK_ID_CURR,TARGET
0,100001,0.5
1,100005,0.5
2,100013,0.5
3,100028,0.5
4,100038,0.5
...,...,...
48739,456221,0.5
48740,456222,0.5
48741,456223,0.5
48742,456224,0.5


In [128]:
sample_submission["TARGET"] = y_predict_lgbm
display(sample_submission)

,SK_ID_CURR,TARGET
0,100001,0.044165
1,100005,0.088767
2,100013,0.017611
3,100028,0.050904
4,100038,0.163890
...,...,...
48739,456221,0.022152
48740,456222,0.049552
48741,456223,0.052747
48742,456224,0.038714


In [129]:
sample_submission.to_csv('dataset/submission6.csv', index=False)

kaggleのスコアは**0.72515**で、ごくわずかなスコアアップという結果となった。